In [12]:
import pandas as pd
from datetime import datetime

def fetch_over_under_runline(oddshark_id, game_date):
    year = game_date.year
    url = f"https://www.oddsshark.com/stats/gamelog/baseball/mlb/{oddshark_id}?season={year}"
    
    try:
        tables = pd.read_html(url)
        df = tables[0]
    except Exception as e:
        print(f"BAD - error for team {oddshark_id} on date {game_date}: {e}")
        return 'unknown', None, None, None
    
    if df.empty:
        print(f"BAD - No data in table for team {oddshark_id} on date {game_date}")
        return 'unknown', None, None, None
    
    df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y')
    matching_rows = df[df['Date'] == game_date]

    if len(matching_rows) > 1:
        print(f"DOUBLEHEADER on {game_date}")
        return '', oddshark_id, year, game_date
    
    if matching_rows.empty:
        print(f"BAD - No matching date found for team {oddshark_id} on date {game_date}")
        return 'unknown', None, None, None
    
    over_under = matching_rows.iloc[0]['Total']
    return over_under, None, None, None

def update_gamelogs_with_over_under(game_pks_file, gamelogs_folder):
    game_pks_df = pd.read_csv(game_pks_file)
    game_pks = game_pks_df['game_id'].tail(50)  # Set the number of recent games to do
    duplicates = []
    
    count = 0
    for game_id in game_pks:
        try:
            gamelog_file = f'{gamelogs_folder}/game_{game_id}.csv'
            gamelog_df = pd.read_csv(gamelog_file)
            
            home_oddshark_id = gamelog_df.loc[0, 'home_oddshark_id']
            game_date_str = gamelog_df.loc[0, 'game_date']
            game_date = datetime.strptime(game_date_str, '%Y-%m-%d')
            
            over_under_runline, duplicate_id, duplicate_year, duplicate_date = fetch_over_under_runline(home_oddshark_id, game_date)
            
            if duplicate_id:
                duplicates.append((duplicate_id, duplicate_year, duplicate_date))
                
            gamelog_df['over_under_runline'] = over_under_runline
             
            gamelog_df.to_csv(gamelog_file, index=False)
            print(f"Updated {gamelog_file} with over/under runline.")
        except Exception as e:
            print(f"Error updating gamelog for game_id {game_id}: {e}")

        count+=1
        if count % 100:
            print(count)
    
    print("\nGames with duplicate dates:")
    for dup in duplicates:
        print(f"Team Oddshark ID: {dup[0]}, Year: {dup[1]}, Date: {dup[2].strftime('%Y-%m-%d')}")

# Update the paths as necessary
game_pks_file = 'game_pks.csv'
gamelogs_folder = 'gamelogs'

# Run the update function
update_gamelogs_with_over_under(game_pks_file, gamelogs_folder)

Updated gamelogs/game_745806.csv with over/under runline.
1
Updated gamelogs/game_746372.csv with over/under runline.
2
Updated gamelogs/game_746775.csv with over/under runline.
3
Updated gamelogs/game_746293.csv with over/under runline.
4
Updated gamelogs/game_745973.csv with over/under runline.
5
Updated gamelogs/game_746213.csv with over/under runline.
6
Updated gamelogs/game_745399.csv with over/under runline.
7
Updated gamelogs/game_747188.csv with over/under runline.
8
Updated gamelogs/game_745322.csv with over/under runline.
9
BAD - No matching date found for team 27019 on date 2024-06-25 00:00:00
Updated gamelogs/game_745157.csv with over/under runline.
10
Updated gamelogs/game_745073.csv with over/under runline.
11
Updated gamelogs/game_746695.csv with over/under runline.
12
Updated gamelogs/game_746456.csv with over/under runline.
13
BAD - No matching date found for team 27019 on date 2024-06-25 00:00:00
Updated gamelogs/game_745157.csv with over/under runline.
14
DOUBLEHEADE